# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

FileNotFoundError: [Errno 2] No such file or directory: 'data/world_bank_projects_less.json'

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## Top 10 countries with most projects

In [ ]:
# Load JSON file
json_df = pd.read_json('world_bank_projects.json')

In [ ]:
# Selecting 10 countries with most projects
json_df.loc[:, ['countryname', 'project_name']].groupby('countryname').count().sort_values('project_name', ascending = False).head(10)

## Top 10 project themes

In [7]:
# Load JSON file
with open('world_bank_projects.json') as json_file:
    json_data = json.load(json_file)

In [16]:
# Extracting major theme column
major_theme = json_normalize(json_data, 'mjtheme_namecode', errors = 'ignore')

# Getting top 10 project themes
mtg = major_theme.groupby('name').count().sort_values(by = 'code', ascending = False)

# major_theme.head(10)

In [17]:
mtg.columns = ['counts']
print(mtg.head(10))

                                              counts
name                                                
Environment and natural resources management     223
Rural development                                202
Human development                                197
Public sector governance                         184
Social protection and risk management            158
Financial and private sector development         130
                                                 122
Social dev/gender/inclusion                      119
Trade and integration                             72
Urban development                                 47


## Missing values filled in

In [18]:
missing_val_ref = json_normalize(json_data, 'mjtheme_namecode', 'id')
print(missing_val_ref)

     code                                          name       id
0       8                             Human development  P129828
1      11                                                P129828
2       1                           Economic management  P144674
3       6         Social protection and risk management  P144674
4       5                         Trade and integration  P145310
5       2                      Public sector governance  P145310
6      11  Environment and natural resources management  P145310
7       6         Social protection and risk management  P145310
8       7                   Social dev/gender/inclusion  P144665
9       7                   Social dev/gender/inclusion  P144665
10      5                         Trade and integration  P144933
11      4      Financial and private sector development  P144933
12      6         Social protection and risk management  P146161
13      6                                                P146161
14      2                

In [19]:
missing_val = missing_val_ref[['code', 'name']]
print(missing_val)

     code                                          name
0       8                             Human development
1      11                                              
2       1                           Economic management
3       6         Social protection and risk management
4       5                         Trade and integration
5       2                      Public sector governance
6      11  Environment and natural resources management
7       6         Social protection and risk management
8       7                   Social dev/gender/inclusion
9       7                   Social dev/gender/inclusion
10      5                         Trade and integration
11      4      Financial and private sector development
12      6         Social protection and risk management
13      6                                              
14      2                      Public sector governance
15      4      Financial and private sector development
16     11  Environment and natural resources man

In [20]:
missing_val = missing_val[missing_val['name'] != '']
print(missing_val)

     code                                          name
0       8                             Human development
2       1                           Economic management
3       6         Social protection and risk management
4       5                         Trade and integration
5       2                      Public sector governance
6      11  Environment and natural resources management
7       6         Social protection and risk management
8       7                   Social dev/gender/inclusion
9       7                   Social dev/gender/inclusion
10      5                         Trade and integration
11      4      Financial and private sector development
12      6         Social protection and risk management
14      2                      Public sector governance
15      4      Financial and private sector development
16     11  Environment and natural resources management
18     10                             Rural development
20      2                      Public sector gov

In [21]:
missing_val = missing_val.sort_values('code').drop_duplicates()
print(missing_val)

     code                                          name
1206    1                           Economic management
673    10                             Rural development
1170   11  Environment and natural resources management
808     2                      Public sector governance
415     3                                   Rule of law
975     4      Financial and private sector development
1384    5                         Trade and integration
1116    6         Social protection and risk management
130     7                   Social dev/gender/inclusion
1348    8                             Human development
513     9                             Urban development


In [24]:
missing_val = missing_val.set_index('code').to_dict()
missing_val = missing_val['name']
print(missing_val)

AttributeError: 'dict' object has no attribute 'set_index'

In [29]:
minds = missing_val_ref.loc[missing_val_ref['name'] == '']

In [30]:
missing_val

{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

In [34]:
# print(minds)
for v in minds.index:
    missing_val_ref.loc[v]['name'] = missing_val[missing_val_ref.loc[v]['code']]

In [35]:
missing_val_ref

,code,name,id
0,8,Human development,P129828
1,11,Environment and natural resources management,P129828
2,1,Economic management,P144674
3,6,Social protection and risk management,P144674
4,5,Trade and integration,P145310
5,2,Public sector governance,P145310
6,11,Environment and natural resources management,P145310
7,6,Social protection and risk management,P145310
8,7,Social dev/gender/inclusion,P144665
9,7,Social dev/gender/inclusion,P144665
